In [2]:
import quaternion
import numpy as np

## Difference in ordering of quaternion multiplication

A small angle approximation of rotation is given by:
$$ q_\Delta = 1 + \frac{\omega_x}{2} \Delta t \hat{i} + \frac{\omega_y}{2} \Delta t \hat{j} + \frac{\omega_z}{2} \Delta t \hat{k} $$

If orientation is represented by quaternion $q$, what is the difference between $q q_\Delta$ and $q_\Delta q$?

In [31]:
# Start with orientation pointing west
q0 = quaternion.Quaternion.axis_angle([0,0,1], np.pi/4)
q0 = q0*quaternion.Quaternion.axis_angle([1.0,0,0], 10*np.pi/180)  # kinda gives away order dependence!
print("10 bank, head -45:", q0.euler_angles()*180/np.pi)

q = q0*quaternion.Quaternion(1.0, 0,0,.5*5*np.pi/180)
print("q0*qdelta order result:", q.euler_angles()*180/np.pi)
q = quaternion.Quaternion(1.0, 0,0 ,.5*5*np.pi/180)*q0  # 5 degree heading change only
print("qdelta*q0 order result:", q.euler_angles()*180/np.pi)

10 bank, head -45: [1.00000000e+01 3.97569335e-16 4.50000000e+01]
q0*qdelta order result: [ 9.98163189 -0.86827412 50.00471213]
qdelta*q0 order result: [10.01894153  0.         50.08033229]


From these two examples, we can conclude that $q q_\Delta$ rotates about body axes whereas $q_\Delta q$ rotates about inertial axes. 

## Rotating coordinate systems

Let's represent a few body configurations and then see if the translations from inertial to body coordinate systems make sense.  The inertial coordinate system is x axis north and the z axis away from the center of the earth.  The body coordinate system is x in the direction of the vehicle travel (roll axis), y along the left wing (pitch axis) and z as yaw.

Let's represent the body as headed west.  The north vector in body coordinates should point along the right wing.

In [12]:
print("Headed west")
q = quaternion.Quaternion.axis_angle(np.array([0,0,1.0]), np.pi/2)
print("Euler angles (roll, pitch, yaw):", q.euler_angles()*180/np.pi)
# A vector pointing north ([1,0,0] in inertial coords), should be along right wing (negative y direction in body coordinates)
north_body = (q.inv()*quaternion.Quaternion.from_vec(np.array([1.0,0,0]))*q).as_ndarray()[1:]
print("North in body coordinates:", north_body)
print()

print("Pitch up 10 degrees")
q = quaternion.Quaternion.axis_angle(np.array([0,1.0,0]), -10*np.pi/180)
print("Euler angles (roll, pitch, yaw):", q.euler_angles()*180/np.pi)
# A vector pointing north ([1,0,0] in inertial coords), should be along right wing (negative y direction in body coordinates)
north_body = (q.inv()*quaternion.Quaternion.from_vec(np.array([1.0,0,0]))*q).as_ndarray()[1:]
print("North in body coordinates:", north_body)
print()

print("Headed west then banked left 20 degrees")
q = quaternion.Quaternion.axis_angle(np.array([0,0,1.0]), np.pi/2)
q = q*quaternion.Quaternion.axis_angle(np.array([1.0,0,0]), -20*np.pi/180)
print("Euler angles (roll, pitch, yaw):", q.euler_angles()*180/np.pi)
# A vector pointing north ([1,0,0] in inertial coords), should be along right wing (negative y direction in body coordinates)
north_body = (q.inv()*quaternion.Quaternion.from_vec(np.array([1.0,0,0]))*q).as_ndarray()[1:]
print("North in body coordinates:", north_body)
print()

print("Heading 315, pitched up 5 degrees, rolled left 20 degrees")
q = quaternion.Quaternion.axis_angle(np.array([0,0,1.0]), np.pi/4)
q = q*quaternion.Quaternion.axis_angle(np.array([0,1.0,0]), -5*np.pi/180)
q = q*quaternion.Quaternion.axis_angle(np.array([1.0,0,0]), -20*np.pi/180)
print("Euler angles (roll, pitch, yaw):", q.euler_angles()*180/np.pi)
# A vector pointing north ([1,0,0] in inertial coords), should be along right wing (negative y direction in body coordinates)
north_body = (q.inv()*quaternion.Quaternion.from_vec(np.array([1.0,0,0]))*q).as_ndarray()[1:]
print("North in body coordinates:", north_body)
longitudinal_inertial = (q*quaternion.Quaternion.from_vec(np.array([1.0,0,0]))*q.inv()).as_ndarray()[1:]
print("longitudinal axis in inertial:", longitudinal_inertial)
print()

Headed west
Euler angles (roll, pitch, yaw): [ 0.  0. 90.]
North in body coordinates: [ 0. -1.  0.]

Pitch up 10 degrees
Euler angles (roll, pitch, yaw): [  0. -10.   0.]
North in body coordinates: [ 0.98480775  0.         -0.17364818]

Headed west then banked left 20 degrees
Euler angles (roll, pitch, yaw): [-20.   0.  90.]
North in body coordinates: [-6.93889390e-18 -9.39692621e-01 -3.42020143e-01]

Heading 315, pitched up 5 degrees, rolled left 20 degrees
Euler angles (roll, pitch, yaw): [-20.  -5.  45.]
North in body coordinates: [ 0.70441603 -0.64338486 -0.29975653]
longitudinal axis in inertial: [0.70441603 0.70441603 0.08715574]



The key conclusion here is:

To convert a vector $v$, which can be represented as $p = 1 + v_x \hat{i} + v_y \hat{j} + v_z \hat{k}$, from inertial coordinates to body coordinates, use:
$$ q^{-1} p q$$
and to convert from body to inertial coordinates use:
$$ q p q^{-1} $$

# Create coordinate system if heading is known

If we know $\psi$, how do we create the resultant coordinate system?  $\psi$ can be determined from orientation quaternion from:
$$ \psi = \mbox{atan2}[2(q_0 q_3 + q_1 q_2),
                         1-2(q_2^2+q_3^2))]
                         $$

In [16]:
q = quaternion.Quaternion.axis_angle(np.array([0,0,1.0]), -np.pi/4)
q *= quaternion.Quaternion.axis_angle(np.array([0,1.0,0]), -5*np.pi/180)
q *= quaternion.Quaternion.axis_angle(np.array([1.0,0,0]), -20*np.pi/180)
print("Euler angles (roll, pitch, heading):", q.euler_angles()*180/np.pi)
print("Extracted Psi (heading):", np.arctan2(2*(q[0]*q[3] + q[1]*q[2]), 1-2*(q[2]**2 + q[3]**2))*180/np.pi)

# Quaternion of just heading rotation
psi = np.arctan2(2*(q[0]*q[3] + q[1]*q[2]), 1-2*(q[2]**2 + q[3]**2))
q_head = quaternion.Quaternion.axis_angle([0,0,1.0], psi)
print(q_head.euler_angles()*180/np.pi)

# Yet another way of doing this (that avoids the arctan), is zeroing out the z component of body axis translated to 
# inertial coords
qi = (q*quaternion.Quaternion.from_vec([1.0,0,0])*q.inv()).as_ndarray()[1:]
print(qi)


Euler angles (roll, pitch, heading): [-20.  -5. -45.]
Extracted Psi (heading): -45.00000000000001
[  0.  -0. -45.]
[ 0.70441603 -0.70441603  0.08715574]


## Centripital acceleration 

Centripital acceleration is given by $a_c = g \tan \phi $.  Let's do some example of translating acceleration due to gravity as well as centrifugal force into body coordinates.

60 degree bank is convenient to use because we know it equates to a load factor of 2g. 

In [22]:
psi = 60*np.pi/180  
q = quaternion.Quaternion.axis_angle(np.array([0,0,1.0]), -np.pi/4)
q *= quaternion.Quaternion.axis_angle(np.array([1.0,0,0]), psi)
print("Euler angles (roll, pitch, heading):", q.euler_angles()*180/np.pi)
print(q.as_ndarray())

# vector, in inertial coords, in direction which ac acts
v_ac = (q*quaternion.Quaternion.from_vec([0,-1.0,0])*q.inv()).as_ndarray()[1:]
v_ac[2] = 0
v_ac /= np.sqrt(v_ac[0]**2 + v_ac[1]**2)

g = 1.0  # normalized g units for now

a = np.array([0,0,g]) + v_ac*g*np.tan(psi)
# Accel in body coordinates
ab = (q.inv()*quaternion.Quaternion.from_vec(a)*q).as_ndarray()[1:]
print(ab)

Euler angles (roll, pitch, heading): [ 60.   0. -45.]
[ 0.80010315  0.46193977 -0.19134172 -0.33141357]
[ 5.55111512e-17 -1.11022302e-16  2.00000000e+00]
